In [2]:
import os

# Cambia la cartella di lavoro nel notebook
os.chdir('/Users/enricozhou/Desktop/abc/src')

# Controlla
print("Nuovo working directory:", os.getcwd())

Nuovo working directory: /Users/enricozhou/Desktop/abc/src


In [7]:
import pandas as pd
import numpy as np
import os
import reddit.data_cleaning.reddit_posts as rp
# from supervised_lda.helpers import tokenize_documents

In [8]:
%pip install data_cleaning

Note: you may need to restart the kernel to use updated packages.


In [35]:
reddit_df = rp.load_reddit(path="/Users/enricozhou/Desktop/abc/dat/reddit/")

In [36]:
reddit_df.head()

,gender,post_text,author,author_flair_text,created_utc,link_id,score,controversiality,gilded,id,subreddit,author_flair_css_class,parent_id,response_text
0,female,if this isn’t accurate then idk what is,be-concerned,15F,1542407736,t3_9xpaoc,60,0,0,e9ufsur,infj,sun,e9ufsur,"Yeah, this is definitely me. Then again, I cou..."
1,female,"i’m in highschool right now. two years ago, my...",be-concerned,15F,1545673571,t3_a940fq,5,0,0,ecgumma,infj,sun,ecgumma,Identify a common interest you have and maybe ...
2,female,[It’s me.](https://imgur.com/a/WDavpAM),INFJen,27F,1525914365,t3_8hz688,3,0,0,dyqcclj,infj,clover,dyqcclj,Aphrodite? Is that you?
3,female,u/User_Simulator u/INFJen,INFJen,27F,1528055310,t3_8o8shk,2,0,0,e01ztf1,infj,clover,e01ztf1,+/u/User_Simulator /u/INFJen
4,female,"It's okay, you too are unique. I'm unique, you...",INFJen,27F,1522538912,t3_88m25u,8,0,0,dwlp1jv,infj,clover,dwlp1jv,[Are we?](https://www.wikihow.com/Stop-Taking-...


In [11]:
# Un flair è un piccolo testo (o un’icona) associato all’autore di un post/commento.
'''
Serve come etichetta visiva, per esempio:
Età + Genere → 25M, 30/F, F/28
Ruolo → Moderator
Categoria → Expert, Beginner
Ogni subreddit può definire i propri flair liberi (inseriti dall’utente) o standardizzati (scelti da un menu).

Nei dati di Reddit, li trovi in due colonne:
author_flair_text → testo libero del flair (es. "28M", "F/29", "Mod")
author_flair_css_class → classe CSS usata nel subreddit per stilizzare il flair (es. "female", "sun", "blue")
'''
reddit_df['op_gender_visible'] = True
reddit_df['op_gender'] = ''
reddit_df.loc[reddit_df.author_flair_css_class.isin(['female']), 'op_gender'] = 'F'
reddit_df.loc[reddit_df.author_flair_css_class.isin(['male']), 'op_gender'] = 'M'

'''
Spiegazione della regex \d F|\dF|\/F

\d → matcha una cifra (es. 2, 5, 9)
\d F → trova pattern tipo "28 F"
\dF → trova pattern tipo "28F" (senza spazio)
\/F → trova /F come in "M/29"
'''
notmale = ~reddit_df.author_flair_text.str.contains('\d M|\dM|\/M', regex=True, na=False)
female = reddit_df.author_flair_text.str.contains('\d F|\dF|\/F', regex=True, na=False)
notfemale = ~reddit_df.author_flair_text.str.contains('\d F|\dF|\/F', regex=True, na=False)
male = reddit_df.author_flair_text.str.contains('\d M|\dM|\/M', regex=True, na=False)
notnull = reddit_df.author_flair_text.notnull()

reddit_df.loc[notnull & female & notmale, 'op_gender'] = 'F'
reddit_df.loc[notnull & male & notfemale, 'op_gender'] = 'M'
# se il flair non è né F né M, lo lasciamo vuoto
reddit_df = reddit_df[reddit_df.op_gender != '']

In [12]:
# codice originale
'''total_counts_by_sub = reddit_df.groupby(['subreddit']).size().reset_index(name='total')
gender_counts_by_sub = reddit_df.groupby(['subreddit', 'op_gender']).size().reset_index(name='count')
gender_counts_by_sub = gender_counts_by_sub.merge(total_counts_by_sub, how='inner', on='subreddit')
gender_counts_by_sub['count'] = gender_counts_by_sub['count']/gender_counts_by_sub['total']'''

# ✅ 1. Conta il numero totale di post per ogni subreddit
total_counts_by_sub = (
    reddit_df                   # DataFrame dove prende i dati
    .groupby(['subreddit'])     # Raggruppa per subreddit
    .size()                     # Conta quante righe (post) ci sono in ogni gruppo
    .reset_index(name='total')  # aggiunge un nuova colonna 'total' con il numero di post in ogni gruppo nel DataFrame
)

# ✅ 2. Conta il numero di post per ogni combinazione (subreddit + genere dell'autore)
gender_counts_by_sub = (
    reddit_df                              # DataFrame dove prende i dati
    .groupby(['subreddit', 'op_gender'])   # Raggruppa per subreddit e genere
    .size()                                # Conta quanti post in ogni combinazione
    .reset_index(name='count')             # aggiunge una nuova colonna 'count' e conta quanti elementi ci sono nel gruppo
)

# ✅ 3. Unisce i due DataFrame per aggiungere la colonna 'total' al conteggio per genere
gender_counts_by_sub = gender_counts_by_sub.merge(
    total_counts_by_sub,     # DataFrame con i totali
    how='inner',             # Join interna (mantiene solo subreddit presenti in entrambi)
    on='subreddit'           # Chiave di unione: subreddit
)

# ✅ 4. Normalizza i conteggi trasformandoli in proporzioni (percentuali)
# Qui 'count' diventa la frazione di post di quel genere sul totale del subreddit
gender_counts_by_sub['count'] = gender_counts_by_sub['count'] / gender_counts_by_sub['total']

# ✅ Ora 'gender_counts_by_sub' contiene:
# - subreddit
# - op_gender (genere dell'autore)
# - count (proporzione di post scritti da quel genere nel subreddit)
# - total (numero totale di post nel subreddit)


In [13]:
gender_counts_by_sub.head()

,subreddit,op_gender,count,total
0,100DaysofKeto,F,0.942529,87
1,100DaysofKeto,M,0.057471,87
2,AskMen,F,0.069311,63597
3,AskMen,M,0.930689,63597
4,AskMenOver30,F,0.055569,12093


In [14]:
threshold = 0.1 # Soglia per filtrare i subreddit con meno del 10% di post di un genere
f = gender_counts_by_sub[gender_counts_by_sub['count'] < threshold] # Filtra i subreddit dove la proporzione di post di un genere è inferiore alla soglia [threshold] (10 %)
# f = f[f.op_gender == 'W']

In [15]:
print(f)

         subreddit op_gender     count  total
1    100DaysofKeto         M  0.057471     87
2           AskMen         F  0.069311  63597
4     AskMenOver30         F  0.055569  12093
7         AskWomen         M  0.055095  51402
9   AskWomenOver30         M  0.083610   6626
23     infertility         M  0.033215  11230
31           proED         M  0.066575   8036
39          xxketo         M  0.000633  12636


In [30]:
total = len(np.unique(reddit_df.subreddit)) # Conta il numero totale di subreddit unici nel DataFrame originale
total

20

In [29]:
reddit_df['op_id'] = reddit_df['author']
reddit_df

,gender,post_text,author,author_flair_text,created_utc,link_id,score,controversiality,gilded,id,subreddit,author_flair_css_class,parent_id,response_text,op_gender_visible,op_gender,sub_id,op_id
0,female,if this isn’t accurate then idk what is,be-concerned,15F,1542407736,t3_9xpaoc,60,0,0,e9ufsur,infj,sun,e9ufsur,"Yeah, this is definitely me. Then again, I cou...",True,F,12,be-concerned
1,female,"i’m in highschool right now. two years ago, my...",be-concerned,15F,1545673571,t3_a940fq,5,0,0,ecgumma,infj,sun,ecgumma,Identify a common interest you have and maybe ...,True,F,12,be-concerned
2,female,[It’s me.](https://imgur.com/a/WDavpAM),INFJen,27F,1525914365,t3_8hz688,3,0,0,dyqcclj,infj,clover,dyqcclj,Aphrodite? Is that you?,True,F,12,INFJen
3,female,u/User_Simulator u/INFJen,INFJen,27F,1528055310,t3_8o8shk,2,0,0,e01ztf1,infj,clover,e01ztf1,+/u/User_Simulator /u/INFJen,True,F,12,INFJen
4,female,"It's okay, you too are unique. I'm unique, you...",INFJen,27F,1522538912,t3_88m25u,8,0,0,dwlp1jv,infj,clover,dwlp1jv,[Are we?](https://www.wikihow.com/Stop-Taking-...,True,F,12,INFJen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422199,male,Has anyone bought a house from someone they ha...,PowerFIRE,"29M, FI@28 with 1M. NW&gt;1.5M. &gt;80% SR. No...",1524234408,t3_8dmr3d,4,0,0,dxokhql,financialindependence,,dxokhql,I would ask in /r/realestate as people there h...,True,M,10,PowerFIRE
422200,male,I've been thinking lately about if/when I want...,PowerFIRE,"29M, FI@28 with 1M. NW&gt;1.5M. &gt;80% SR. No...",1516724485,t3_7sdov2,8,0,0,dt4asco,financialindependence,,dt4asco,You can do both. Fill your tax-free space with...,True,M,10,PowerFIRE
422201,male,"Well, my networth is down something like 150k ...",PowerFIRE,"30M, FI@28 with 1M. NW&gt;1.5M. &gt;80% SR. No...",1545837351,t3_a9nqwa,22,0,0,ecl84ws,financialindependence,,ecl84ws,"Oh, sure. Try ERN's article on variable withdr...",True,M,10,PowerFIRE
422202,male,"That's a lot of cash, and doubly so if you are...",PowerFIRE,"30M, FI@28 with 1M. NW&gt;1.5M. &gt;80% SR. No...",1544181026,t3_a3wexr,118,0,0,eba4uha,financialindependence,,eba4uha,My 'Quick-math' says 4% of the 2.2 mil he has ...,True,M,10,PowerFIRE


In [31]:
gender_visible = reddit_df[reddit_df.op_gender_visible == True]
gv_stats = gender_visible.groupby(['subreddit', 'op_gender', 'op_id']).size().reset_index(name='counts')
len(np.unique(gv_stats.op_id))

29498

In [33]:
gv_stats

,subreddit,op_gender,op_id,counts
0,100DaysofKeto,F,FlashKat,4
1,100DaysofKeto,F,LurkingRaeven,2
2,100DaysofKeto,F,Luxtos2111,4
3,100DaysofKeto,F,Medeavi,2
4,100DaysofKeto,F,MiaowMiaowMeow,1
...,...,...,...,...
32345,xxketo,F,zbanned,2
32346,xxketo,M,ajc506,1
32347,xxketo,M,diamund223,4
32348,xxketo,M,super_soprano13,1


In [ ]:
ops = reddit_df.groupby(['op_id', 'subreddit']).size().reset_index(name='counts') # Conta il numero di post per ogni autore (op_id) in ogni subreddit
total = len(np.unique(ops.op_id))
ask_men = ops[ops.subreddit == 'AskMen']
ask_men_ops = len(np.unique(ask_men.op_id))
print(ask_men_ops/total, ask_men_ops, total)

'''
✅ Stampa:
La frazione di autori totali che hanno scritto in AskMen
Il numero di autori unici di AskMen
Il numero totale di autori unici
'''

0.20648857549664384 6091 29498


In [34]:
ops

,op_id,subreddit,counts
0,---ChumbaWumba,loseit,2
1,--Chimaera--,fatlogic,38
2,--Chimaera--,loseit,1
3,--Poot--,AskWomen,2
4,--mackie--,loseit,3
...,...,...,...
32334,zyphax,loseit,35
32335,zyrnil,keto,11
32336,zzaannsebar,loseit,2
32337,zzzuluz,AskMenOver30,1
